In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import re
import csv
import os
import gc
import torch
# set to "cuda:1" for running in parallel on both GPUs
device = torch.device("cuda:1")
torch.cuda.set_device(device)
torch.set_default_device(device)
import Stemmer
import pandas as pd
from tqdm import tqdm
from utils.graph import KGraphPreproc
from utils.graph.chain import GraphChain
from utils.llm.mistral import MistralLLM
from utils.prompt import GRAPH_QA_PROMPT, ENTITY_PROMPT
from utils.file import export_results_to_file

In [3]:
def get_response(prompt):
    # global chain
    # del mistral
    gc.collect()
    torch.cuda.empty_cache()
    r = chain.invoke(prompt)
    return r["result"]


def save_results(fpath, data_rows):
    with open(fpath, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["Model"])
        for r in data_rows:
            writer.writerow([str(r)])

In [4]:
mistral = MistralLLM()

In [5]:
def get_fbqa_data(question_row):
    """
    Takes in a dataset row and returns Q and A as strings
    """
    question = question_row.get("RawQuestion", None)
    return question, None

In [6]:
####### load the graph
fbkb_graph = KGraphPreproc.get_fbkb_graph()

In [7]:
chain = GraphChain.from_llm(
    llm=mistral,
    graph=fbkb_graph,
    qa_prompt=GRAPH_QA_PROMPT,
    entity_prompt=ENTITY_PROMPT,
    verbose=False,
)
chain.sbert_cache_path = "/datasets/FB15k-237/cache/sbert.csv"

In [8]:
# load q's
fbqa = pd.read_csv("/datasets/FreebaseQA/FbQA-eval-1000.csv", index_col=0)
fbqa.head(1)

,Question-ID,RawQuestion,ProcessedQuestion,Parses,entities
2,FreebaseQA-eval-2,Who directed the films; The Fisher King (1991)...,who directed the films; the fisher king (1991)...,"[{'Parse-Id': 'FreebaseQA-eval-2.P0', 'Potenti...","[['/m/07j6w', '/m/07h5d'], ['/m/04z257', '/m/0..."


In [ ]:
for depth in [1,2,3]:
    print(f"depth: {depth}")
    # set the depth
    chain.exploration_depth = depth
    # init experiment
    experiment_name = f"sbert-kb{depth}"
    res_path = f"/datasets/FreebaseQA/results/{experiment_name}.csv"
    results = []
    id_list = []
    l = 0
    # load if preinit'ed
    if os.path.isfile(res_path):
        r_df = pd.read_csv(res_path)
        l = len(r_df)
        results = list(r_df.Model.values)
    # run through
    for c, (i, r) in enumerate(tqdm(list(fbqa.iterrows()))):
        id_list.append(i)
        if c < l:
            continue
        q, a = get_fbqa_data(r)
        response = get_response(q)
        results.append(response)
        # backup every 10 qs
        if c % 10 == 0:
            export_results_to_file(res_path, results, id_list)
    export_results_to_file(res_path, results)

depth: 1


100%|██████████| 1000/1000 [00:00<00:00, 2080507.94it/s]

depth: 2


  0%|          | 0/1000 [00:00<?, ?it/s]/model/utils/graph/chain.py:111: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  entity_string = self.entity_extraction_chain.run(question)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Checking embedding cache
Loading embedding cache


329136it [11:51, 462.61it/s]
/model/utils/graph/chain.py:148: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain(
100%|██████████| 1000/1000 [4:00:54<00:00, 14.45s/it] 


depth: 3


 81%|████████  | 807/1000 [13:38:41<2:33:29, 47.72s/it]

## MQA

In [13]:
metaqa_graph = KGraphPreproc.get_metaqa_graph()

chain = GraphChain.from_llm(
    llm=mistral,
    graph=metaqa_graph,
    qa_prompt=GRAPH_QA_PROMPT,
    entity_prompt=ENTITY_PROMPT,
    verbose=False,
)
chain.sbert_cache_path = "/datasets/MetaQA/cache/sbert.csv"


In [1]:
for hop in ["1hop", "2hop", "3hop"]:
    print(hop)
    # load q's
    metaqa = pd.read_csv(f"/datasets/MetaQA/{hop}/test_1000.txt", header=None, index_col=0)
    metaqa.rename(columns={1: "Question", 2: "Answers"}, inplace=True)


    for depth in [2]:
        print(f"depth: {depth}")
        # set the depth
        chain.exploration_depth = depth
        # init experiment
        experiment_name = f"sbert-kb{depth}"
        res_path = f"/datasets/MetaQA/results/{hop}/{experiment_name}.csv"
        results = []
        id_list = []
        l = 0
        # load if preinit'ed
        if os.path.isfile(res_path):
            r_df = pd.read_csv(res_path)
            l = len(r_df)
            results = list(r_df.Model.values)
        # run through
        for c, (i, r) in enumerate(tqdm(list(metaqa.iterrows()))):
            id_list.append(i)
            if c < l:
                continue
            q = r.Question
            response = get_response(q)
            results.append(response)
            # backup every 10 qs
            if c % 10 == 0:
                export_results_to_file(res_path, results, id_list)
        export_results_to_file(res_path, results, id_list)

1hop


NameError: name 'pd' is not defined